# Практическая работа № 5 «Тайные многосторонние вычисления и разделение секрета»
 
В практической работе необходимо привести последовательность выполнения следующих протоколов:
- тайных многосторонних вычислений для расчета средней величины трех чисел. В качестве исходных данных принять коды 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите;
- разбиения секрета с использование гаммирования для трех участников. В качестве секрета принять первые 3 буквы фамилии, для гамм - любые трехбуквенные сочетания;
- разделения секрета по схеме Шамира для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите;
- разделения секрета по схеме Асмута-Блума для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите.

При оформлении отчета необходимо привести исходные данные и таблицы, содержащие последовательность выполнения протоколов. Отчет необходимо загрузить в edu.susu.ru


In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
with open('instance/settings.json') as f:
    settings = json.load(f)
    LAST_NAME = settings['last_name']
    FIRST_NAME = settings['first_name']
    PATRONYMIC = settings['patronymic']

In [3]:
def power_with_modulo(a, n, m):
    res = 1
    while n > 0:
        if (n & 1) == 1:
            res = (res * a) % m
        a = (a * a) % m
        n >>= 1
    return res

## Тайные многосторонние вычисления

In [4]:
def rsa_generator():
    keys = [
        [5, 91, 29],
        [7, 91, 31],
        [17, 91, 17],
    ]
    for e, n, d in keys:
        yield {'public': {'e': e, 'n': n}, 'private': {'d': d}}

In [5]:
smc_data = list(map(lambda c: ord(c) - ord('a') + 1, LAST_NAME[:3]))
smc_data = {k: {'secret': v, 'rsa': rsa} for k, v, rsa in zip('abc', smc_data, rsa_generator())}
print(json.dumps(smc_data, indent=4))

{
    "a": {
        "secret": 1,
        "rsa": {
            "public": {
                "e": 5,
                "n": 91
            },
            "private": {
                "d": 29
            }
        }
    },
    "b": {
        "secret": 12,
        "rsa": {
            "public": {
                "e": 7,
                "n": 91
            },
            "private": {
                "d": 31
            }
        }
    },
    "c": {
        "secret": 11,
        "rsa": {
            "public": {
                "e": 17,
                "n": 91
            },
            "private": {
                "d": 17
            }
        }
    }
}


<b>A</b> прибавляет случайное секретное число $x$ к своему секрету, шифрует результат с помощью открытого ключа <b>B</b> и отсылает его <b>B</b>

In [6]:
np.random.seed(42)
a_x = np.random.randint(2, 1<<20)

a_data = smc_data['a']['secret'] + a_x
shared_data = power_with_modulo(
    a_data, 
    smc_data['b']['rsa']['public']['e'], 
    smc_data['b']['rsa']['public']['n'],
)

{{f'$secret_a = {smc_data["a"]["secret"]}$'}}  
{{f'$x_a = {a_x}$'}}  
{{f'$data_a = {a_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

<b>B</b> расшифровывает результат, добавляет к нему свою зарплату, шифрует результат с помощью открытого ключа <b>C</b> и отсылает его <b>C</b>.

In [7]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['b']['rsa']['private']['d'], 
    smc_data['b']['rsa']['public']['n'],
)
b_data = decrypted_data + smc_data['b']['secret']
shared_data = power_with_modulo(
    b_data, 
    smc_data['c']['rsa']['public']['e'], 
    smc_data['c']['rsa']['public']['n'],
)

{{f'$secret_b = {smc_data["b"]["secret"]}$'}}  
{{'$data_{decrypted} = %d$' % decrypted_data}}  
{{f'$data_b = {b_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

In [8]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['c']['rsa']['private']['d'], 
    smc_data['c']['rsa']['public']['n'],
)
c_data = decrypted_data + smc_data['c']['secret']
shared_data = power_with_modulo(
    c_data, 
    smc_data['a']['rsa']['public']['e'], 
    smc_data['a']['rsa']['public']['n'],
)

{{f'$secret_c = {smc_data["c"]["secret"]}$'}}  
{{'$data_{decrypted} = %d$' % decrypted_data}}  
{{f'$data_c = {c_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

In [9]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['a']['rsa']['private']['d'], 
    smc_data['a']['rsa']['public']['n'],
)
final_data = (decrypted_data - a_x) % smc_data['a']['rsa']['public']['n']

{{'$data_{decrypted} = %d$' % decrypted_data}}   
{{'$data_{final} = %d$' % final_data}}  
{{f'$secret_a + secret_b + secret_c = {smc_data["a"]["secret"] + smc_data["b"]["secret"] +smc_data["c"]["secret"]}$'}}   

## Разбиение секрета с использование гаммирования

In [21]:
ssg_data = LAST_NAME[:3]
print(ssg_data)

alk


Кодирование секрета

In [31]:
def get_binary(word):
    return list(map(lambda c: '{0:0>8b}'.format(c), word.encode('cp1251')))

In [35]:
pd.DataFrame({
    'Символ': list(ssg_data),
    'Бинарный код': get_binary(ssg_data)
}).set_index('Символ').T

Символ,a,l,k
Бинарный код,01100001,01101100,01101011


In [39]:
def get_gamma(size, seed):
    random = np.random.RandomState(seed)
    res = ''
    for i in range(size):
        res += chr(ord('a') + random.randint(0, 26))
    return res

In [64]:
def get_gammas(n, size):
    names = [f'Гамма{i + 1}' for i in range(n)]
    values = [get_gamma(size, i) for i in range(n)]
    bin_values = [get_binary(v) for v in values]
    values = list(map(list , values))
    values_title =  [f'{g} Символ {i + 1}' for i, g in enumerate(names)]
    res = {}
    for i, name in enumerate(names):
        res[f'{name} Символ'] = values[i]
        res[f'{name} Бинарный код'] = bin_values[i]
    return res

Генерация случайных гамм и передача их участникам.

In [68]:
ssg_df = pd.DataFrame({
    'Символ': list(ssg_data),
    'Бинарный код': get_binary(ssg_data),
    **get_gammas(3, len(ssg_data))
}).set_index('Символ').T
ssg_df

Символ,a,l,k
Бинарный код,01100001,01101100,01101011
Гамма1 Символ,m,p,v
Гамма1 Бинарный код,01101101,01110000,01110110
Гамма2 Символ,f,l,m
Гамма2 Бинарный код,01100110,01101100,01101101
Гамма3 Символ,i,p,n
Гамма3 Бинарный код,01101001,01110000,01101110


Получение шифрограммы и выкладывание её в доступное для участников место.

In [144]:
def get_cipher(data):
    res = []
    for row in data:
        n = len(row[0])
        cur = np.zeros((n,), dtype=int)
        for i in range(n):
            for v in row:
                cur[i] = (cur[i] + int(v[i])) % 2
        res.append(''.join(map(str, cur)))   
    return np.array(res)

In [145]:
cipher = get_cipher(ssg_df.values[::2].T)
cipher

array(['00000011', '00000000', '00011110'], dtype='<U8')

In [146]:
ssgc_df = ssg_df.T.assign(Шифрограммы=cipher).T
ssgc_df

Символ,a,l,k
Бинарный код,01100001,01101100,01101011
Гамма1 Символ,m,p,v
Гамма1 Бинарный код,01101101,01110000,01110110
Гамма2 Символ,f,l,m
Гамма2 Бинарный код,01100110,01101100,01101101
Гамма3 Символ,i,p,n
Гамма3 Бинарный код,01101001,01110000,01101110
Шифрограммы,00000011,00000000,00011110


Восстановление кода

In [152]:
gammas = ssg_df.values[2::2]
gammas

array([['01101101', '01110000', '01110110'],
       ['01100110', '01101100', '01101101'],
       ['01101001', '01110000', '01101110']], dtype=object)

In [153]:
cipher

array(['00000011', '00000000', '00011110'], dtype='<U8')

In [197]:
bin_decodes = get_cipher(np.vstack((gammas, cipher)).T)
bin_decodes

array(['01100001', '01101100', '01101011'], dtype='<U8')

In [199]:
decodes = list(map(lambda x: int(x, 2), get_cipher(np.vstack((gammas, cipher)).T)))
decodes

[97, 108, 107]

In [204]:
bytes(decodes).decode('cp1251')

'alk'